In [29]:
#IMPORT THESE PACKAGES
import selenium
from selenium import webdriver
import pandas as pd
#OPTIONAL PACKAGE, BUY MAYBE NEEDED
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import os
from selenium.webdriver.common.keys import Keys
import urllib.parse

import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.proxy import Proxy, ProxyType

import random

In [30]:
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = os.environ.get("GOOGLE_CHROME_BIN", "C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe")
    # Initialize WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [31]:
def log_driver(driver):
    USERNAME="@ghorbel"
    PASSWORD= "......"
    driver.get("https://twitter.com/login")

# Wait for page to load
    time.sleep(8)

# Locate the username input field and enter username
    username_input = driver.find_element(By.XPATH, '//*[@autocomplete="username"]')
    username_input.send_keys(USERNAME)
    username_input.send_keys(Keys.RETURN)

# Wait for password field to appear
    time.sleep(2)

# Locate the password input field and enter password
    password_input = driver.find_element(By.XPATH, '//*[@autocomplete="current-password"]')
    password_input.send_keys(PASSWORD)
    password_input.send_keys(Keys.RETURN)

    time.sleep(3)  # Small delay to ensure page fully loads


In [32]:
def scrape_trends(driver):
    log_driver(driver)

    driver.get("https://x.com/explore/tabs/trending")
    time.sleep(5)

    trends = driver.find_elements(By.XPATH, '//div[@data-testid="trend"]//div[@style="color: rgb(231, 233, 234);"]//span')

    while not trends:
        log_driver(driver)
        driver.get("https://x.com/explore/tabs/trending")
        time.sleep(10)
        trends = driver.find_elements(By.XPATH, '//div[@data-testid="trend"]//div[@style="color: rgb(231, 233, 234);"]//span')

    X_trends = set()

    for trend in trends:
        X_trends.add(trend.text)
    encoded_texts = {urllib.parse.quote(text) for text in X_trends}
    data1 = {'Trend': [], 'Tweet': []}
    fulldf = pd.DataFrame(data1)
    counter = 0
    for trend in encoded_texts:
        driver.get("https://x.com/search?q={}&src=trend_click&vertical=trends".format(trend))
        time.sleep(4)
        tweet_texts = set()
        scroll_pause_time = 5  # Adjust if needed
        d_trend = urllib.parse.unquote(trend)
        def scroll_down():
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(scroll_pause_time)

# Loop to keep scrolling
        max_scrolls = 5  # Adjust based on how many tweets you want
        for _ in range(max_scrolls):
    
        # Find all tweet text elements
            tweets = driver.find_elements(By.XPATH, '//*[@data-testid="tweetText"]')
            if not tweets:
                print("No tweets found!")
                return fulldf
            
    
        # Extract text and add to the set (to avoid duplicates)
            for tweet in tweets:
                row = [d_trend, tweet.text]
                fulldf.loc[len(fulldf)] = row
        # Scroll down
            scroll_down()

        tweets = driver.find_elements(By.XPATH, '//*[@data-testid="tweetText"]')
        for tweet in tweets:
            tweet_texts.add(tweet)
            row = [d_trend, tweet.text]
            fulldf.loc[len(fulldf)] = row

    return fulldf

In [33]:
fulldf = scrape_trends(driver)

No tweets found!


In [38]:
fulldf

,Trend,Tweet
0,Argentina,Igor Serrote reminding Argentina who has 5 stars
1,Argentina,Una pareja de pingüinos de Magallanes se acurr...
2,Argentina,I GOT RUGGED BY $LIBRA \n\nTHESE POOR COUNTRIE...
3,Argentina,"This was $20,000 few minutes ago $Libra \n\n..."
4,Argentina,Argentina's President Milei Launched the $LIBR...
...,...,...
458,Raquel,Oh my god. That wasn't an edited video. They'v...
459,Raquel,E quando o Dourado fizer um VT com a voz do Di...
460,Raquel,"Beatriz certíssima! Juliano não VALE NADA, e a..."
461,Raquel,Don’t worry guys. Liv and Raquel may on the vi...


In [40]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

def upload_to_google_sheet(df, sheet_name, credentials_file):
    # Define the scope
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    # Add your service account credentials
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)

    # Authorize the clientsheet 
    client = gspread.authorize(creds)

    # Create a new Google Sheet or open an existing one
    try:
        sheet = client.create(sheet_name)
    except gspread.exceptions.APIError:
        sheet = client.open(sheet_name)

    # Select the first sheet
    worksheet = sheet.get_worksheet(0)

    # Update the sheet with the DataFrame
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

# Example usage
upload_to_google_sheet(fulldf, "X Trends", "D:\Downloads\poised-legend-451008-j5-0c9e2ed6e1e8.json")